# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Shakhun'ya,100,RU,1602639313,92,57.68,46.61,50.85,6.98
1,Vaini,40,TO,1602638942,69,-21.20,-175.20,75.20,13.87
2,Ketchikan,40,US,1602639149,81,55.34,-131.65,48.20,10.29
3,Puerto Ayora,45,EC,1602638944,90,-0.74,-90.35,67.93,9.10
4,Brae,51,GB,1602638986,81,60.40,-1.35,50.00,16.11
...,...,...,...,...,...,...,...,...,...
571,Asău,71,RO,1602639000,79,46.43,26.40,46.99,5.99
572,San Juan del Sur,100,NI,1602639375,86,11.25,-85.87,78.48,8.52
573,Ternate,100,ID,1602639375,71,0.80,127.40,83.98,4.21
574,Nara,20,JP,1602639376,64,34.69,135.80,75.99,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
66,Gunnedah,0,AU,1602639319,25,-30.98,150.25,75.20,5.82
74,Dubbo,0,AU,1602639320,36,-32.25,148.62,77.00,8.05
112,Faanui,0,PF,1602638948,73,-16.48,-151.75,79.00,9.44
133,Tura,0,IN,1602639326,76,25.52,90.22,79.75,1.81
145,Grand-Santi,0,GF,1602639327,78,4.25,-54.38,76.30,3.51
146,Salalah,0,OM,1602639000,64,17.02,54.09,75.20,3.36
159,Vao,0,NC,1602639202,84,-22.67,167.48,74.03,4.92
258,Mangla,0,PK,1602639338,34,31.89,72.38,74.46,2.46
260,Jauharabad,0,PK,1602639339,32,32.29,72.28,76.01,5.08
266,Conceição do Araguaia,0,BR,1602639339,68,-8.26,-49.26,78.53,4.03


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
66,Gunnedah,AU,-30.98,150.25,
74,Dubbo,AU,-32.25,148.62,
112,Faanui,PF,-16.48,-151.75,
133,Tura,IN,25.52,90.22,
145,Grand-Santi,GF,4.25,-54.38,
146,Salalah,OM,17.02,54.09,
159,Vao,NC,-22.67,167.48,
258,Mangla,PK,31.89,72.38,
260,Jauharabad,PK,32.29,72.28,
266,Conceição do Araguaia,BR,-8.26,-49.26,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 66: Gunnedah.
Closest hotel in Gunnedah is Golden Chain Overlander Motor Lodge.
------------
Retrieving Results for Index 74: Dubbo.
Closest hotel in Dubbo is Comfort Inn Blue Lagoon.
------------
Retrieving Results for Index 112: Faanui.
Closest hotel in Faanui is InterContinental Bora Bora Resort Thalasso Spa.
------------
Retrieving Results for Index 133: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
------------
Retrieving Results for Index 145: Grand-Santi.
Closest hotel in Grand-Santi is Hotel La Villa Couacou.
------------
Retrieving Results for Index 146: Salalah.
Closest hotel in Salalah is Alpha Suites Hotel.
------------
Retrieving Results for Index 159: Vao.
Closest hotel in Vao is OURE TERA beach resort.
------------
Retrieving Results for Index 258: Mangla.
Closest hotel in Mangla is Bismillah milk shop.
------------
Retrieving Results for Index 260: Jauharabad.
Closest hotel in Jauharabad is Gulzar Hotel & Restaurant.
------------
Retriev

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
66,Gunnedah,AU,-30.98,150.25,Golden Chain Overlander Motor Lodge
74,Dubbo,AU,-32.25,148.62,Comfort Inn Blue Lagoon
112,Faanui,PF,-16.48,-151.75,InterContinental Bora Bora Resort Thalasso Spa
133,Tura,IN,25.52,90.22,Hotel Polo Orchid
145,Grand-Santi,GF,4.25,-54.38,Hotel La Villa Couacou
146,Salalah,OM,17.02,54.09,Alpha Suites Hotel
159,Vao,NC,-22.67,167.48,OURE TERA beach resort
258,Mangla,PK,31.89,72.38,Bismillah milk shop
260,Jauharabad,PK,32.29,72.28,Gulzar Hotel & Restaurant
266,Conceição do Araguaia,BR,-8.26,-49.26,Tarumã Tropical Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))